In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 1T\[MT10] Series de Tiempo\Clases prácticas\Práctica 1-20250419\uncovered.xlsx"
df = pd.read_excel(path)
df.head()

,dateid01,dateid,dep,dif,e,libor,tbill
0,1978-07-01,1978-09-30 23:59:59.999,NaN,-0.53,1.8497,6.82,6.29
1,1978-10-01,1978-12-31 23:59:59.999,0.024436,-2.01,1.8949,9.02,7.01
2,1979-01-01,1979-03-31 23:59:59.999,0.059423,-1.67,2.0075,9.66,7.99
3,1979-04-01,1979-06-30 23:59:59.999,-0.001096,-2.28,2.0053,11.63,9.35
4,1979-07-01,1979-09-30 23:59:59.999,0.034010,-3.20,2.0735,12.66,9.46


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   dateid01  121 non-null    datetime64[ns]
 1   dateid    121 non-null    datetime64[ns]
 2   dep       120 non-null    float64       
 3   dif       121 non-null    float64       
 4   e         121 non-null    float64       
 5   libor     121 non-null    float64       
 6   tbill     121 non-null    float64       
dtypes: datetime64[ns](2), float64(5)
memory usage: 6.7 KB


In [3]:
df['dateid01'] = pd.to_datetime(df['dateid01'])
df['dateidQ'] = df['dateid01'].dt.to_period('Q')
df.sort_values(by=['dateid01'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   dateid01  121 non-null    datetime64[ns]
 1   dateid    121 non-null    datetime64[ns]
 2   dep       120 non-null    float64       
 3   dif       121 non-null    float64       
 4   e         121 non-null    float64       
 5   libor     121 non-null    float64       
 6   tbill     121 non-null    float64       
 7   dateidQ   121 non-null    period[Q-DEC] 
dtypes: datetime64[ns](2), float64(5), period[Q-DEC](1)
memory usage: 7.7 KB


In [4]:
df_f = df[['dateid', 'dep', 'dif']].dropna()
df_f.set_index('dateid', inplace=True)

from statsmodels.tsa.api import VAR

# Crear el modelo VAR.
model = VAR(df_f, freq='QE')
var_results = model.fit(maxlags=2)
print(var_results.summary())

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Fri, 02, May, 2025
Time:                     17:09:23
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -5.58929
Nobs:                     118.000    HQIC:                  -5.72876
Log likelihood:           18.7523    FPE:                 0.00295577
AIC:                     -5.82410    Det(Omega_mle):      0.00272035
--------------------------------------------------------------------
Results for equation dep
            coefficient       std. error           t-stat            prob
-------------------------------------------------------------------------
const         -0.003884         0.007354           -0.528           0.597
L1.dep         0.115925         0.098136            1.181           0.237
L1.dif        -0.004421         0.004114           -1.075           0.282
L2.dep        -0

In [5]:
import statsmodels.api as sm

# Supongamos que df tiene las tres columnas: y1, y2, y3
# Y queremos un VAR con p rezagos
p = 2  # número de rezagos

# Crear las variables rezagadas
df_lagged = pd.concat([df_f.shift(i) for i in range(1, p+1)], axis=1)
df_lagged.columns = [f'{col}_lag{i}' for i in range(1, p+1) for col in df_f.columns]

# DataFrame final: quitamos los primeros p registros (por el rezago)
df_model = pd.concat([df_f, df_lagged], axis=1).dropna()

# Variables endógenas (Y) y variables explicativas (X)
Y = df_model[df_f.columns]
X = df_model.drop(columns=df_f.columns)
X = sm.add_constant(X)  # agregar intercepto

results = {}

for col in Y.columns:
    model = sm.OLS(Y[col], X).fit()
    n = model.nobs  # número de observaciones
    k = model.df_model + 1  # número de parámetros (incluyendo intercepto)

    aic_manual = (2 * k - 2 * model.llf) / n
    bic_manual = (np.log(n) * k - 2 * model.llf) / n

    results[col] = {
        'R2': model.rsquared,
        'Adj_R2': model.rsquared_adj,
        'Sum_sq_resids': np.sum(model.resid ** 2),
        'SE_equation': np.sqrt(np.sum(model.resid ** 2) / model.df_resid),
        'F-statistic': model.fvalue,
        'LogLikelihood': model.llf,
        'AIC': aic_manual,
        'BIC': bic_manual,
        'Mean_dependent': np.mean(Y[col]),
        'SD_dependent': np.std(Y[col], ddof=1)
    }

pd.DataFrame(results)

,dep,dif
R2,0.036608,0.749754
Adj_R2,0.002505,0.740896
Sum_sq_resids,0.260980,147.559395
SE_equation,0.048058,1.142732
F-statistic,1.073467,84.638900
LogLikelihood,193.290967,-180.623970
AIC,-3.191372,3.146169
BIC,-3.073970,3.263571
Mean_dependent,0.001048,-2.814153
SD_dependent,0.048118,2.244951


In [ ]:
# Controlar tabla de arriba

In [12]:
def eviews_style_stats(var_result):
    """
    Emula exactamente la salida de EViews para un VAR de statsmodels.
    Calcula determinantes ajustados y no ajustados, log likelihood, AIC, BIC y número de coeficientes.
    """
    n_obs = var_result.nobs
    n_eqs = var_result.neqs
    p = var_result.k_ar

    # Número de parámetros estimados por ecuación
    k_per_eq = n_eqs * p + 1

    # Matriz de residuos
    resid = var_result.resid

    # Covarianza de residuos (ajustada y no ajustada)
    sigma_u_adj = (resid.T @ resid) / (n_obs - k_per_eq)
    sigma_u_unadj = (resid.T @ resid) / n_obs

    # Determinantes
    det_sigma_u_adj = np.linalg.det(sigma_u_adj)
    det_sigma_u = np.linalg.det(sigma_u_unadj)

    # Log-likelihood
    loglik = var_result.llf

    # Número total de parámetros
    k_total = n_eqs * (n_eqs * p + 1)

    # AIC y BIC
    aic_var = (2 * k_total - 2 * loglik) / n_obs
    bic_var = (np.log(n_obs) * k_total - 2 * loglik) / n_obs

    # Mostrar resultados
    print(f"{'Determinant resid covariance (dof adj.)':40s} {det_sigma_u_adj:.6f}")
    print(f"{'Determinant resid covariance':40s} {det_sigma_u:.6f}")
    print(f"{'Log likelihood':40s} {loglik:.4f}")
    print(f"{'Akaike information criterion':40s} {aic_var:.6f}")
    print(f"{'Schwarz criterion':40s} {bic_var:.6f}")
    print(f"{'Number of coefficients':40s} {k_total}")

eviews_style_stats(var_results)

Determinant resid covariance (dof adj.)  0.002720
Determinant resid covariance             0.002495
Log likelihood                           18.7523
Akaike information criterion             -0.148344
Schwarz criterion                        0.086460
Number of coefficients                   10


In [13]:
import numpy as np
import pandas as pd
from statsmodels.tsa.api import VAR
from scipy.stats import chi2

def select_var_eviews_exact(data, maxlags=8, trend='c'):
    """
    Reproduce la tabla VAR de EViews con:
      • criterios ajustados (AIC, SC, HQ, FPE) marcando el mínimo global
      • LR* secuencial modificado marcando el primer rechazo H₀ (5%)
    """
    k      = data.shape[1]
    T_orig = data.shape[0]
    T_max  = T_orig - maxlags
    const  = k * (1 + np.log(2*np.pi))
    crit_lr = chi2.ppf(0.95, k**2)

    # 1) Estimaciones y captura de llf, fpe, aic0, bic0, hq0, SSR
    llf, fpe, aic0, bic0, hq0, SSR = {}, {}, {}, {}, {}, {}
    for p in range(maxlags+1):
        data_p = data.iloc[maxlags-p:]
        res    = VAR(data_p, freq='QE').fit(p, trend=trend)
        llf[p]    = res.llf
        fpe[p]    = res.fpe
        aic0[p]   = res.aic
        bic0[p]   = res.bic
        hq0[p]    = res.hqic
        SSR[p]    = res.resid.T.dot(res.resid)

    # 2) Construcción de la tabla sin marcas
    rows = []
    for p in range(maxlags+1):
        # criterios ajustados
        AIC = aic0[p] + const
        SC  = bic0[p] + const
        HQ  = hq0[p]  + const

        # LR* secuencial modificado
        if p == 0:
            LR = np.nan
        else:
            _, ld0 = np.linalg.slogdet(SSR[p-1] / T_max)
            _, ld1 = np.linalg.slogdet(SSR[p]   / T_max)
            m       = 1 + k*p
            LR      = (T_max - m) * (ld0 - ld1)

        rows.append({
            'Lag':  p,
            'Nobs': T_max,
            'LogL': llf[p],
            'LR':   LR,
            'FPE':  fpe[p],
            'AIC':  AIC,
            'SC':   SC,
            'HQ':   HQ
        })

    df = pd.DataFrame(rows)

    # 0) Convertimos a object las columnas que vamos a stringificar
    for col in ['LR','FPE','AIC','SC','HQ']:
        df[col] = df[col].astype(object)

    # 3) Marcar mínimos globales para FPE, AIC, SC, HQ
    for crit in ['FPE','AIC','SC','HQ']:
        idx = df[crit].idxmin()
        df.loc[idx, crit] = f"{df.loc[idx,crit]:.6f}*"

    # 4) Marcar primer rechazo H₀ en LR* (descendiendo desde p=maxlags)
    #    según chi2(0.95, k^2)
    for p in range(maxlags, 0, -1):
        val = df.loc[df['Lag']==p, 'LR']
        # si aún es numérico y supera el crítico, marcamos y rompemos
        if isinstance(val.values[0], float) and val.values[0] > crit_lr:
            df.loc[df['Lag']==p, 'LR'] = f"{val.values[0]:.6f}*"
            break

    # 5) Formatear las columnas numéricas a 6 decimales
    for col in ['LogL','LR','FPE','AIC','SC','HQ']:
        df[col] = df[col].apply(lambda x: f"{x:.6f}" if isinstance(x, float) else x)

    return df[['Lag','Nobs','LogL','LR','FPE','AIC','SC','HQ']]

# Ejemplo de uso:
tabla = select_var_eviews_exact(df_f, maxlags=8, trend='c')
print(tabla.to_string(index=False))

 Lag  Nobs       LogL          LR       FPE        AIC         SC         HQ
   0   112 -65.909913         nan  0.011527   1.212677   1.261222   1.232373
   1   112  15.511113 158.480211* 0.002893* -0.169841* -0.024207* -0.110753*
   2   112  17.543977    3.884223  0.002996  -0.134714   0.108009  -0.036233
   3   112  19.063104    2.848362  0.003133  -0.090413   0.249400   0.047460
   4   112  19.656633    1.091669  0.003330  -0.029583   0.407319   0.147682
   5   112  21.960877    4.155869  0.003435   0.000699   0.534689   0.217356
   6   112  22.705315    1.316060  0.003643   0.058834   0.689914   0.314883
   7   112  27.354262    8.052641  0.003606   0.047245   0.775415   0.342687
   8   112  30.705579    5.685269  0.003653   0.058829   0.884088   0.393663


In [14]:
# Crear el modelo VAR.
model = VAR(df_f, freq='QE')
var_results1 = model.fit(maxlags=1)
print(var_results1.summary())

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Fri, 02, May, 2025
Time:                     00:26:23
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -5.72037
Nobs:                     119.000    HQIC:                  -5.80360
Log likelihood:           16.9922    FPE:                 0.00284989
AIC:                     -5.86050    Det(Omega_mle):      0.00271145
--------------------------------------------------------------------
Results for equation dep
            coefficient       std. error           t-stat            prob
-------------------------------------------------------------------------
const         -0.003711         0.007145           -0.519           0.603
L1.dep         0.129073         0.094315            1.369           0.171
L1.dif        -0.001806         0.002037           -0.887           0.375

Results for equ